In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pathlib import Path
import pandas as pd
import pprint
import re
import time
from inltk.inltk import get_sentence_similarity

file = open("/kaggle/input/tamilstopwords/tamil-stop-words.txt")
tamil_stop_words = file.read().split("\n")
file.close()

file = open("/kaggle/input/hindistopwords/hindi-stop-words.txt")
hindi_stop_words = file.read().split("\n")
file.close()

def jaccard(str1, str2): 
  """ text similarity metric """
  a = set(str1.lower().split()) 
  b = set(str2.lower().split())
  c = a.intersection(b)
  return float(len(c)) / (len(a) + len(b) - len(c))

def is_tamil_stop_word(word):
  """ returns true if word is a Tamil stop word, false otherwise """
  return word in tamil_stop_words

def is_hindi_stop_word(word):
  """ returns true if word is a Hindi stop word, false otherwise """
  return word in hindi_stop_words

def remove_stop_words(word,language):
  """ removes all Tamil stop words from list """
  if language == "tamil":
    return list(filter(lambda x: not is_tamil_stop_word(x), word))
  else:
    return list(filter(lambda x: not is_hindi_stop_word(x), word))

def predict(data):
  """ predicts answers to questions """
  answers = []
  for index, row in data.iterrows():
    ID       = row['id']
    context  = row['context']
    question = row['question']
    language = row['language']
    context  = context.split('.')
    best_score    = 0
    best_sentence ='-'
    for s in context:
      s = s.strip()
      s = s.split(' ')
      s = remove_stop_words(s,language)
      s = ' '.join(s) 
      # if jaccard(s, question) > best_score:
      #   best_score    = jaccard(s, question)
      #   best_sentence = s
      if get_sentence_similarity(s,question,language[:2]) > best_score:
        best_score     = get_sentence_similarity(s,question,language[:2])
        best_sentencce = s
    answers.append(best_sentence)
  return answers 

if __name__ == "__main__":
  #train = pd.read_csv("/home/hofer/Kaggle/data/train.csv")
  test  = pd.read_csv("/kaggle/input/chaii-hindi-and-tamil-question-answering/test.csv")
  answers = predict(test)
  submission = pd.DataFrame({'id': test.id, 'PredictionString': answers})
  submission.to_csv('submission.csv', index=False)
